In [3]:
%matplotlib notebook
import time
import sklearn
import pandas as pd
import numpy as np
import category_encoders
from numpy.linalg.linalg import LinAlgError
from collections import Counter
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression as lr
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix as confu
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.cluster import KMeans
from itertools import combinations as Cb


In [4]:
# Normalize data
def normalization(training_numpy, testing_numpy):
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy)
    testing_x_scaled = min_max.transform(testing_numpy)
    #training_norm = pd.DataFrame(x_scaled, columns = columns)
    #testing_norm = pd.DataFrame(testting_x_scaled, columns = columns)
    return (x_scaled, testing_x_scaled)

def pca_function(rate, data):
    pca = PCA()
    pca.fit(data)
    for thres_n in xrange(1,len(data)):
        if sum(pca.explained_variance_ratio_[:thres_n])>rate:
            pca_n = thres_n
            break
    
    pca = PCA(n_components=pca_n)
    pca_data = pca.fit_transform(data)
    return (pca_data, pca)

def lda_whole_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


def lda_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        print np.unique(original_y)
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


'''
def pca_function(n, data):
    pca = PCA(n_components=n)
    pca_data = pca.fit_transform(data)
    return (pca_data.flatten(), pca)

def lda_function(X, y):
    lda = LDA()
    lda_data = lda.fit_transform(X, y)
    return (lda_data.flatten(), lda)
'''
def replace_value(training_normal_numpy, key, loc):  
    training_normal_numpy[loc][key] = 1
    
def del_zero_column(df_train, df_test):
    new_df_test = pd.DataFrame(np.zeros([df_test.shape[0], df_train.shape[1]]),columns=df_train.columns)
    new_df_test[list(set(df_train.columns) & set(df_test.columns))] = df_test[list(set(df_train.columns) & set(df_test.columns))]
    #df_test = df_test[df_train.columns]
    return (df_train, new_df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]
#def del_zero_column(df_train, df_test):
#    df_train = df_train.loc[:, (df_train != 0).any(axis=0)]
#    df_test = df_test[df_train.columns]
#    return (df_train, df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]

def all_lda(train_combine, label_num, n_cluster = 2):
    train_all_dummy = pd.get_dummies(train_combine)
    #train_all_dummy = category_encoders.OneHotEncoder(cols=train_combine.columns.tolist()).fit_transform(train_combine)
    (train_lda, lda) = lda_whole_function(train_all_dummy, np.array(label_num), n_cluster=n_cluster)
    clf=BernoulliNB()
    start = time.time()
    
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score , time.time()-start)

def divide_lda(train_combine, label_num, n_cluster = 2):
    train_column= train_combine.columns
    for col in train_combine.columns:
        train_dummy.append(pd.get_dummies(train_combine[col]))
        #train_dummy.append(category_encoders.OneHotEncoder(cols=[col]).fit_transform(train_combine[[col]]))
    train_lda = pd.DataFrame(index=range(len(label_num)))
    for i in range(len(train_combine.columns)):
        #print train_dummy[i].shape
        (now_train_lda, lda) = lda_whole_function(train_dummy[i], np.array(label_num), n_cluster=3)
        now_lda_col_name = []
        for j in range(len(now_train_lda[0])):
            train_lda[train_column[i]+'_'+str(j+1)] = np.array(now_train_lda)[:,j]
    start = time.time()
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score, time.time()-start)

# DNA

In [5]:
train=pd.read_csv('splice.data.txt', header=None)
train.columns=['label', 'name', 'dna']
key_str = {}
for (i,label_name) in zip(range(len(np.unique(train['label']))), np.unique(train['label'])):
    key_str[label_name] = i
    
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
train_dummy = []
    
train['dna'] = train['dna'].map(lambda x: list(str(x).strip()))
for idx in xrange(60):
    train['dna_%d'% (idx,)] = train['dna'].map(lambda x: x[idx])

train_column = train.columns[3:]
train = train[train_column]

result = pd.DataFrame(columns=['score', 'time'])


total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    


train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for cluster in range(2,5):
    for iteration in range(5):
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:    
            (train_lda,lda) = lda_whole_function(train_dummy.iloc[train_val_index], val_label[train_val_index], n_cluster=cluster)        
            clf.fit(train_lda, val_label[train_val_index])
            test_lda = lda.transform(train_dummy.iloc[test_val_index])
            start = time.time()
            now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
            val_time.append(time.time() - start)         
        exe_time = np.sum(val_time)
        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "cluster:", cluster
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(cluster)] = [np.average(total_score), np.average(total_time)]

    result.to_csv('dna_lda_kmeans.csv')

/usr/anaconda2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/anaconda2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


cluster: 2
score: 0.883683197919
time: 0.00990114212036
cluster: 3
score: 0.87433398374
time: 0.00768890380859
cluster: 4
score: 0.871999435616
time: 0.012050930659


# Mushroom

In [6]:
mush_columns = ['label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
               'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',\
               'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type',\
               'spore-print-color', 'population', 'habitat']
train = pd.read_csv('agaricus-lepiota.data',header=None)
train.columns = mush_columns
key_str = {'e':0, 'p':1}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['score', 'time'])


total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    


train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for cluster in range(2,5):
    for iteration in range(5):
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:    
            (train_lda,lda) = lda_whole_function(train_dummy.iloc[train_val_index], val_label[train_val_index], n_cluster=cluster)        
            clf.fit(train_lda, val_label[train_val_index])
            test_lda = lda.transform(train_dummy.iloc[test_val_index])
            start = time.time()
            now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
            val_time.append(time.time() - start)         
        exe_time = np.sum(val_time)
        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "cluster:", cluster
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(cluster)] = [np.average(total_score), np.average(total_time)]

    result.to_csv('mush_lda_kmeans.csv')

cluster: 2
score: 0.956922269593
time: 0.0204949855804
cluster: 3
score: 0.909261628677
time: 0.0162106990814
cluster: 4
score: 0.904765986785
time: 0.0151782989502


# Car

In [7]:
car_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
train = pd.read_csv('car.data',header=None)
train.columns = car_columns
key_str = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['score', 'time'])

total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    


train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for cluster in range(2,5):
    for iteration in range(5):
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:    
            (train_lda,lda) = lda_function(train_dummy.iloc[train_val_index], val_label[train_val_index], n_cluster=cluster)        
            #print train_lda.shape
            clf.fit(train_lda, val_label[train_val_index])
            test_lda = lda.transform(train_dummy.iloc[test_val_index])
            start = time.time()
            now_score.append(accuracy_score(clf.predict(test_lda), val_label[test_val_index]))
            val_time.append(time.time() - start)         
        exe_time = np.sum(val_time)       
        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "cluster:", cluster
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(cluster)] = [np.average(total_score), np.average(total_time)]

    result.to_csv('car_lda_kmeans.csv')

cluster: 2
score: 0.671025732691
time: 0.0108588218689
cluster: 3
score: 0.666246265146
time: 0.0127231836319
cluster: 4
score: 0.662913166814
time: 0.0109859466553


# Botnet

In [8]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_5.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['score', 'time'])
total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    


train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for cluster in range(2,5):
    for iteration in range(5):
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:               
            (train_lda,lda) = lda_whole_function(train_dummy.iloc[train_val_index], val_label[train_val_index], n_cluster=cluster) 
            train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
            test_lda = lda.transform(train_dummy.iloc[test_val_index])
            test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
            (train_lda, test_lda) = normalization(train_lda, test_lda)
                 
            clf.fit(train_lda, val_label[train_val_index])
            start = time.time()
            now_score.append(f1_score(clf.predict(test_lda), val_label[test_val_index]))
            val_time.append(time.time() - start)         
        exe_time = np.sum(val_time)      
        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "cluster:", cluster
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(cluster)] = [np.average(total_score), np.average(total_time)]

    result.to_csv('bot5_lda_kmeans.csv')

/usr/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


cluster: 2
score: 0.0875542984573
time: 0.0373637199402
cluster: 3
score: 0.154456256111
time: 0.0322838068008
cluster: 4
score: 0.123737895992
time: 0.0312878449758


In [9]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_6.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['score', 'time'])
total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    


train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for cluster in range(2,5):
    for iteration in range(5):
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:               
            (train_lda,lda) = lda_whole_function(train_dummy.iloc[train_val_index], val_label[train_val_index], n_cluster=cluster) 
            train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
            test_lda = lda.transform(train_dummy.iloc[test_val_index])
            test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
            (train_lda, test_lda) = normalization(train_lda, test_lda)
                 
            clf.fit(train_lda, val_label[train_val_index])
            start = time.time()
            now_score.append(f1_score(clf.predict(test_lda), val_label[test_val_index]))
            val_time.append(time.time() - start)         
        exe_time = np.sum(val_time)      
        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "cluster:", cluster
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(cluster)] = [np.average(total_score), np.average(total_time)]

    result.to_csv('bot6_lda_kmeans.csv')

cluster: 2
score: 0.566807834502
time: 0.0312148571014
cluster: 3
score: 0.578790922071
time: 0.0285457611084
cluster: 4
score: 0.576957980091
time: 0.0302193959554


In [10]:
column_str = ['Proto', 'State','Dir','Dport']
column_numeric = ['Dur', 'sTos', 'dTos', 'TotPkts', 'TotBytes','SrcBytes']
train = pd.read_csv('botnet_7.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])

train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]
train_numeric = train[column_numeric]
train = train[column_str]

result = pd.DataFrame(columns=['score', 'time'])
total_score = []
total_time = []
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    


train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for cluster in range(2,5):
    for iteration in range(5):
        val_time = []
        now_score = []
        for train_val_index, test_val_index in skf:               
            (train_lda,lda) = lda_whole_function(train_dummy.iloc[train_val_index], val_label[train_val_index], n_cluster=cluster) 
            train_lda = np.append(train_lda, train_numeric.iloc[train_val_index].values, axis=1)
            test_lda = lda.transform(train_dummy.iloc[test_val_index])
            test_lda = np.append(test_lda, train_numeric.iloc[test_val_index].values, axis=1)        
            (train_lda, test_lda) = normalization(train_lda, test_lda)
                 
            clf.fit(train_lda, val_label[train_val_index])
            start = time.time()
            now_score.append(f1_score(clf.predict(test_lda), val_label[test_val_index]))
            val_time.append(time.time() - start)         
        exe_time = np.sum(val_time)      
        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "cluster:", cluster
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(cluster)] = [np.average(total_score), np.average(total_time)]

    result.to_csv('bot7_lda_kmeans.csv')

cluster: 2
score: 0.0364285714286
time: 0.0350560188293
cluster: 3
score: 0.167946608947
time: 0.036266040802
cluster: 4
score: 0.231040885041
time: 0.0346938610077
